In [20]:
import cv2
import numpy as np

In [21]:
srcs = ["unebouteille", "troisbouteilles", "bouteilleetverre"]
imgs = []
bws = []
for src in srcs:
    img = cv2.imread("C:/DEV/traitement d'images/findbottle/" + src + ".jpg")
    (thresh, bw) = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
    print(thresh)
    imgs.append(img)
    bws.append(bw)
    
print(len(imgs))

127.0
127.0
127.0
3


In [22]:
t = 0

for i in imgs:
    cv2.imshow("image" + str(t), i)
    t += 1
    
for bwi in bws:
    cv2.imshow("bw" + str(t), bwi)
    t += 1

cv2.waitKey(0)

-1

In [23]:
img_canny = cv2.Canny(imgs[0], 200, 200)
cv2.imshow("canny", img_canny)
cv2.waitKey(0)

-1

In [24]:
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv2.filter2D(bws[0], -1, kernelx)
img_prewitty = cv2.filter2D(bws[0], -1, kernely)

cv2.imshow("prewittx", img_prewittx)
cv2.imshow("prewitty", img_prewitty)
cv2.imshow("prewittx+y", img_prewittx+img_prewitty)
cv2.waitKey(0)

-1

In [25]:
img_sobelx = cv2.Sobel(bws[0],cv2.CV_8U,1,0,ksize=5)
img_sobely = cv2.Sobel(bws[0],cv2.CV_8U,0,1,ksize=5)
img_sobel = img_sobelx + img_sobely

cv2.imshow("sobel", img_sobel)
cv2.waitKey(0)

-1

In [26]:
img_laplacian = cv2.Laplacian(bws[0], cv2.CV_64F)

cv2.imshow("laplacian", img_laplacian)
cv2.waitKey(0)

-1

In [42]:
new_img = cv2.imread("C:/DEV/traitement d'images/findbottle/" + srcs[2] + ".jpg")
gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
(new_thresh, new_bw) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

cv2.imshow("bw", new_bw)
cv2.waitKey(0)

-1

In [43]:
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(new_bw, cv2.MORPH_OPEN, kernel, iterations = 2)
sure_bg = cv2.dilate(opening,kernel,iterations=3)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

cv2.imshow("opening", opening)
cv2.imshow("bg", sure_bg)
cv2.imshow("fg", sure_fg)
cv2.imshow("unknown", unknown)
cv2.waitKey(0)

-1

In [44]:
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers+1
markers[unknown==255] = 0

In [45]:
markers = cv2.watershed(new_img,markers)
new_img[markers == -1] = [255,0,0]

In [46]:
cv2.imshow("new_img", new_img)
cv2.waitKey(0)

-1

In [49]:
import cv2
import numpy as np

filename = 'chessboard.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

In [50]:
filename = 'chessboard.jpg'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# find Harris corners
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
dst = cv2.dilate(dst,None)
ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

cv2.imwrite('subpixel5.png',img)

True

In [51]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

MIN_MATCH_COUNT = 10

img1 = cv2.imread('bottlenobg.png',0)          # queryImage
img2 = cv2.imread('diningbottle.png',0) # trainImage

# Initiate SIFT detector
sift = cv2.SIFT()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

AttributeError: module 'cv2.cv2' has no attribute 'SIFT'